# Filter-Experimente

Wir verwenden verschiedene 3×3- und 5×5-Kernel:
- Blur (Weichzeichnen)
- Horizontale und vertikale Kanten (Edge Detection)
- Sobel-Filter
- Emboss (Relief-Effekt)
- Diagonale Kanten

Am Ende könnt ihr euren **eigenen Filter** ausprobieren!

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color, io

## 1. Faltungsfunktion

In [ ]:
def simple_convolution(gray_image, kernel):
    kh, kw = kernel.shape
    ih, iw = gray_image.shape
    oh = ih - kh + 1
    ow = iw - kw + 1
    output = np.zeros((oh, ow), dtype=float)
    for i in range(oh):
        for j in range(ow):
            patch = gray_image[i:i+kh, j:j+kw]
            output[i, j] = np.sum(patch * kernel)
    return output


## 3. Beispielbild laden
Wir verwenden die bekannte `camera`-Aufnahme als Graustufenbild.

In [ ]:
image_gray = data.camera().astype(float) / 255.0
plt.figure(figsize=(4, 4))
plt.imshow(image_gray, cmap='gray')
plt.title('Original (Graustufen)')
plt.axis('off')
plt.show()

## 4. Verschiedene Filter definieren

Hier sind einige **vorgefertigte Filter**, mit denen ihr spielen könnt.


In [ ]:
# Blur (Weichzeichnen)
kernel_blur = np.ones((3, 3)) / 9.0

# Horizontale Kanten
kernel_edge_h = np.array([[-1, -1, -1],
                          [ 0,  0,  0],
                          [ 1,  1,  1]])

# Vertikale Kanten
kernel_edge_v = np.array([[-1,  0,  1],
                          [-1,  0,  1],
                          [-1,  0,  1]])

# Sobel (horizontal)
kernel_sobel_x = np.array([[-1, 0, 1],
                           [-2, 0, 2],
                           [-1, 0, 1]])

# Sobel (vertikal)
kernel_sobel_y = np.array([[-1, -2, -1],
                           [ 0,  0,  0],
                           [ 1,  2,  1]])

# Emboss (Relief-Effekt)
kernel_emboss = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])

# Diagonale Kante (↘)
kernel_diag1 = np.array([[ 0, -1, -1],
                         [ 1,  0, -1],
                         [ 1,  1,  0]])

kernels = {
    'Blur (3x3)': kernel_blur,
    'Edge horizontal': kernel_edge_h,
    'Edge vertikal': kernel_edge_v,
    'Sobel X': kernel_sobel_x,
    'Sobel Y': kernel_sobel_y,
    'Emboss': kernel_emboss,
    'Diagonal ↘': kernel_diag1,
}
list(kernels.keys())

## 5. Alle Filter auf einmal ausprobieren

Wir wenden alle definierten Filter auf das Bild an und vergleichen die Ergebnisse nebeneinander.

In [ ]:
n = len(kernels)
plt.figure(figsize=(4*(n+1), 4))

# Original
plt.subplot(1, n+1, 1)
plt.imshow(image_gray, cmap='gray')
plt.title('Original')
plt.axis('off')

for idx, (name, kernel) in enumerate(kernels.items(), start=2):
    filtered = simple_convolution(image_gray, kernel)
    # Für die Darstellung normalisieren
    f_min, f_max = filtered.min(), filtered.max()
    filtered_show = (filtered - f_min) / (f_max - f_min + 1e-8)
    plt.subplot(1, n+1, idx)
    plt.imshow(filtered_show, cmap='gray')
    plt.title(name)
    plt.axis('off')

plt.tight_layout()
plt.show()

## Was fällt euch auf?

## 6. Eigene Filter ausprobieren

Jetzt seid ihr dran! Definiert eigene 3×3-Filter und schaut, was passiert.

**Ideen:**
- Diagonale Kanten anders herum
- Stärkere Schärfung
- Noch weichere Unschärfe (größere Kernel)


In [ ]:
# Beispiel: eigener 3x3-Filter
my_kernel = np.array([[0, 0, 0,],
                      [0, 1, 0],
                      [0, 0, 0]])
print('Mein Kernel:')
print(my_kernel)

filtered = simple_convolution(image_gray, my_kernel)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(image_gray, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(filtered, cmap='gray', vmin=0, vmax=1)
plt.title('Gefiltertes Bild')
plt.axis('off')
plt.tight_layout()
plt.show()

## Eigenes Bild aus dem Internet verwenden

In der nachfolgenden Zelle könnt ihr ein **eigenes Bild von einer URL** laden (z.B. ein Gebäude, ein Tier, eine Stadt) und einen der oben definierten Filter darauf anwenden.

1. Sucht euch eine Bild-URL (am besten ein `.jpg` oder `.png`).
2. Wählt einen Filternamen aus dem `kernels`-Dictionary (z.B. `"Blur (3x3)"`, `"Emboss"`, `"Sobel X"`). In der nachfolgenden Zelle könnt ihr euren eigenen Kernel definieren, den ihr durch die Angabe "mein_kernel" verwenden könnt! 
3. Führt die Zelle aus und schaut euch das Ergebnis an.

In [ ]:
my_kernel = np.array([[0, 0, 0],
                      [0, 1, 0],
                      [0, 0, 0]])

my_kernel = np.array([[15, 15, 15],
                      [15, 15, 0],
                      [0, 0, 0]])

kernels.update({"mein_kernel":my_kernel})

In [ ]:
def apply_filter_from_url(url, kernel_name="Blur (3x3)", normalize=True):
    """
    Lädt ein Bild von einer URL, wandelt es in Graustufen um
    und wendet einen der definierten Filter darauf an.
    """
    if kernel_name not in kernels:
        print(f"Unbekannter Filtername: {kernel_name}")
        print("Verfügbare Filter:", list(kernels.keys()))
        return

    # Bild laden
    image = io.imread(url)

    # In Graustufen umwandeln (falls RGB-Bild)
    if image.ndim == 3:
        img_gray = color.rgb2gray(image)  # Werte 0..1
    else:
        # Falls schon Graustufen
        img_gray = image.astype(float)
        if img_gray.max() > 1.0:
            img_gray = img_gray / 255.0

    kernel = kernels[kernel_name]

    # Faltung anwenden (hier ohne Padding; du kannst auch simple_convolution_padded verwenden)
    filtered = simple_convolution(img_gray, kernel)

    # Für die Darstellung normalisieren
    if normalize:
        f_min, f_max = filtered.min(), filtered.max()
        filtered_show = (filtered - f_min) / (f_max - f_min + 1e-8)
    else:
        filtered_show = filtered

    # Plot: Original + Gefiltertes Bild
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(img_gray, cmap="gray")
    plt.title("Original (Graustufen)")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(filtered_show, cmap="gray", vmin=0, vmax=1)
    plt.title(f"Gefiltert: {kernel_name}")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
IMAGE_URL = "https://www.berlin.de/sen/sbw/_assets/stadtdaten/geoportal/landesvermessung/tierheim_800.jpg"

apply_filter_from_url(IMAGE_URL, kernel_name="mein_kernel", normalize=True)

### Aufgaben für euch

1. Erstellt einen **stärkeren Blur-Filter**, z.B. 5×5, und vergleicht ihn mit dem 3×3-Blur.
2. Baut einen eigenen **diagonalen Kantenfilter** (↙ statt ↘).
3. Was passiert, wenn **"normalize=True"** durch **"normalize=False"** ersetzt wird?
4. Überlegt: Was passiert, wenn die Summe aller Werte im Kernel sehr groß oder sehr klein ist?